In [1]:
library(Seurat)
library(Signac)
library(parallel)
library(GenomicRanges)

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, 

In [ ]:
# trying to figure out why 

# Error: Can't extract columns that don't exist.
# x Column ranges doesn't exist.

#https://github.com/stuart-lab/signac/issues/221

In [63]:
read_objects <- function(ipath1, ipath2=NULL){
    objs <- list.files(ipath1)[c(1,3)]
    obj_list <- c()

    for(o in objs){
        temp_obj <- readRDS(paste0(ipath1,o))
        rownames(temp_obj@meta.data) <- temp_obj$BARCODE
        obj_list <- append(obj_list, temp_obj)
    }

    if (is.null(ipath2) == FALSE){
        objs2 <- list.files(ipath2)
        for(o in objs2){
        temp_obj2 <- readRDS(paste0(ipath2,o))
        obj_list <- append(obj_list, temp_obj2)
        }
    }

    return(obj_list)
}


In [71]:

# Prepare objects for merging
merge_prep <- function(sobj){
    #### Extract fragments as frags objects from each dataset
    frags <- Fragments(sobj)
    #### Built feature matrices
    counts <- FeatureMatrix(
      fragments = frags,
      features = combined.peaks,
      cells = rownames(sobj@meta.data)
    )
    print(head(rownames(sobj@meta.data)))
    #print(counts )
    #### Create Chromatin assays.. obviously
    # for the multiome, originally did min.features = -1 for each individual obj
    # orig only min.features = -1 for lung pool 4 
    # min.features = 1 min.cells = 1 doesn't work
    assay <- CreateChromatinAssay(counts, fragments = frags, min.features = -1)
    #### Create new object from each assay
    # ******* change project to orig.ident if appropriate *******
    sobj[["ATAC_comb"]] <- assay
    return(sobj)
}



In [74]:
# NOTE may need to change
extract_samp_nm <- function(obs_list){
    samp_nms <- c()
    for (o in obs_list) {
        samp_nm <- as.character(o@meta.data$orig.ident[1])
        samp_nms <- append(samp_nms, samp_nm)
    }
    return(samp_nms)
}



In [59]:
list.files(ipath1, full.names = T)[c(1,3)]

[1] "/home/rlancione/ps-epigen/users/rlan/Donor_Demultiplexing/Objects//QY_2225_2_QY_2224_2_DF.0.05.RDS"
[2] "/home/rlancione/ps-epigen/users/rlan/Donor_Demultiplexing/Objects//QY_2227_2_QY_2226_2_DF.0.05.RDS"

In [64]:

args = commandArgs(trailingOnly=TRUE)
ipath1 <- "/home/rlancione/ps-epigen/users/rlan/Donor_Demultiplexing/Objects/"
outp <- "/oasis/tscc/scratch/cmiciano/Lung/lungmap_3/04_multiome_merge_8/"
proj <- "231108_merged_multiome_lung8"
# min.features = -1 important 
# for cells with 0 features, end up in infinite loop without this if the cell numbers are different between rna and atac
#ipath1 <- "/projects/ps-epigen/users/cmiciano/SenNet_Multiome/07_DoubletFinder_Rep2/df_objs_md_liver/"

Sys.time()
li <- read_objects(ipath1=ipath1)
Sys.time()


[1] "2023-11-08 15:41:43 PST"

[1] "2023-11-08 15:42:17 PST"

In [65]:
li

[[1]]
An object of class Seurat 
179768 features across 20000 samples within 3 assays 
Active assay: SCT (30316 features, 3000 variable features)
 2 other assays present: RNA, ATAC
 2 dimensional reductions calculated: pca, umap

[[2]]
An object of class Seurat 
181455 features across 20000 samples within 3 assays 
Active assay: SCT (30386 features, 3000 variable features)
 2 other assays present: RNA, ATAC
 2 dimensional reductions calculated: pca, umap


In [66]:

# change default assay to atac
# assign to default cellranger ATAC peaks for each object, so the peaks are actually used when calling featurematrix
for (obj in 1:length(li)) {
   print(li[[obj]])
   DefaultAssay(li[[obj]]) <- 'ATAC'

}

for (obj in 1:length(li)) {
   print(li[[obj]])
}

An object of class Seurat 
179768 features across 20000 samples within 3 assays 
Active assay: SCT (30316 features, 3000 variable features)
 2 other assays present: RNA, ATAC
 2 dimensional reductions calculated: pca, umap
An object of class Seurat 
181455 features across 20000 samples within 3 assays 
Active assay: SCT (30386 features, 3000 variable features)
 2 other assays present: RNA, ATAC
 2 dimensional reductions calculated: pca, umap
An object of class Seurat 
179768 features across 20000 samples within 3 assays 
Active assay: ATAC (112851 features, 0 variable features)
 2 other assays present: RNA, SCT
 2 dimensional reductions calculated: pca, umap
An object of class Seurat 
181455 features across 20000 samples within 3 assays 
Active assay: ATAC (114468 features, 0 variable features)
 2 other assays present: RNA, SCT
 2 dimensional reductions calculated: pca, umap


In [67]:
### create list of genomic ranges. Change to "objects"
granges <- mclapply(mc.cores=detectCores()*.75, X= li, FUN=Signac::granges)


### concatonate them
grang.c <- c()


for (g in granges){grang.c <- append(grang.c, g)}
### Create a unified set of peaks to quantify in each dataset


combined.peaks <- reduce(grang.c)
### create proprocessed merged object list




In [68]:
combined.peaks

GRanges object with 131874 ranges and 0 metadata columns:
             seqnames          ranges strand
                <Rle>       <IRanges>  <Rle>
       [1]       chr1      9770-10693      *
       [2]       chr1   180707-181632      *
       [3]       chr1   183940-184768      *
       [4]       chr1   186352-187268      *
       [5]       chr1   191056-191950      *
       ...        ...             ...    ...
  [131870] KI270728.1 1792047-1792755      *
  [131871] KI270731.1       4442-5400      *
  [131872] KI270734.1   116901-117898      *
  [131873] KI270734.1   121024-121924      *
  [131874] KI270734.1   163727-164630      *
  -------
  seqinfo: 37 sequences from an unspecified genome; no seqlengths

In [69]:
li[[1]]@meta.data #barcodes added back in as rownames

,BARCODE,orig.ident,nCount_RNA,nFeature_RNA,nCount_ATAC,nFeature_ATAC,fragments_freq_count,FRiP,percent.mt,nCount_SCT,⋯,BEST.POSTERIOR,SNG.POSTERIOR,SNG.BEST.GUESS,SNG.BEST.LLK,SNG.NEXT.GUESS,SNG.NEXT.LLK,SNG.ONLY.POSTERIOR,DBL.BEST.GUESS,DBL.BEST.LLK,DIFF.LLK.SNG.DBL
,<chr>,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
AAACAGCCAAACGGGC-1,AAACAGCCAAACGGGC-1,QY_2225_2_QY_2224_2,6581,3072,5854,2926,9272,0.6313632,1.10925391,6172,⋯,-2.5e+02,1,UCSDX51_UCSDX51,-243.32,D371_D371,-574.97,0,"D381_D381,UCSDX51_UCSDX51,0.50",-301.66,58.34
AAACAGCCAAAGGTAC-1,AAACAGCCAAAGGTAC-1,QY_2225_2_QY_2224_2,6785,2512,2357,1209,3871,0.6088866,3.38983051,6013,⋯,-1.8e+02,1,D265_D265,-173.03,UCSDX3_UCSDX3,-291.65,0,"D265_D265,UCSDX3_UCSDX3,0.50",-182.59,9.56
AAACAGCCAAATACCT-1,AAACAGCCAAATACCT-1,QY_2225_2_QY_2224_2,773,607,2335,1213,3731,0.6258376,1.16429495,4272,⋯,-7.2e+01,1,UCSDX19_UCSDX19,-68.06,UCSDX51_UCSDX51,-145.11,0,"UCSDX19_UCSDX19,UCSDX31_UCSDX31,0.50",-79.20,11.14
AAACAGCCAATTGACT-1,AAACAGCCAATTGACT-1,QY_2225_2_QY_2224_2,1709,1121,8280,4112,20682,0.4003481,2.57460503,4407,⋯,-4.0e+02,1,UCSDX42_UCSDX42,-391.82,UCSDX40_UCSDX40,-956.91,0,"UCSDX40_UCSDX40,UCSDX42_UCSDX42,0.50",-486.25,94.43
AAACAGCCACATAGCC-1,AAACAGCCACATAGCC-1,QY_2225_2_QY_2224_2,5791,2752,84,43,81,1.0370370,0.41443619,5755,⋯,-5.0e+01,1,D339_D339,-45.57,UCSDX2_UCSDX2,-129.87,0,"D339_D339,D371_D371,0.50",-62.54,16.96
AAACAGCCACCAGCAT-1,AAACAGCCACCAGCAT-1,QY_2225_2_QY_2224_2,13831,4553,7542,3804,9703,0.7772854,0.62179163,6228,⋯,5.0e-207,1,UCSDX44_UCSDX44,-625.03,UCSDX19_UCSDX19,-680.52,0,"UCSDX19_UCSDX19,UCSDX44_UCSDX44,0.50",-468.25,-156.78
AAACAGCCACTCAACA-1,AAACAGCCACTCAACA-1,QY_2225_2_QY_2224_2,6616,2807,5308,2637,6761,0.7850910,0.19649335,6142,⋯,-2.2e+02,1,D265_D265,-217.55,D283_D283,-551.49,0,"D265_D265,D381_D381,0.50",-281.11,63.56
AAACAGCCAGAGGGAG-1,AAACAGCCAGAGGGAG-1,QY_2225_2_QY_2224_2,255,222,2382,1220,8488,0.2806315,3.13725490,4128,⋯,-1.5e+02,1,D335_D335,-149.09,UCSDX21_UCSDX21,-363.05,0,"D335_D335,UCSDX21_UCSDX21,0.50",-183.51,34.42
AAACAGCCAGGCCTTG-1,AAACAGCCAGGCCTTG-1,QY_2225_2_QY_2224_2,11851,4666,8238,4034,16879,0.4880621,1.56948781,6629,⋯,1.5e-264,1,UCSDX31_UCSDX31,-689.13,D339_D339,-860.94,0,"D339_D339,UCSDX31_UCSDX31,0.50",-600.68,-88.45


In [72]:

Sys.time()
pmo_list <- lapply( X= li, FUN=merge_prep)
Sys.time()


[1] "2023-11-08 15:44:18 PST"

Extracting reads overlapping genomic regions



[1] "AAACAGCCAAACGGGC-1" "AAACAGCCAAAGGTAC-1" "AAACAGCCAAATACCT-1"
[4] "AAACAGCCAATTGACT-1" "AAACAGCCACATAGCC-1" "AAACAGCCACCAGCAT-1"


Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from atac_comb_ to ataccomb_”
Extracting reads overlapping genomic regions



[1] "AAACAGCCAAATACCT-1" "AAACAGCCAATTGAGA-1" "AAACAGCCACCTCGCT-1"
[4] "AAACAGCCACTAAGCC-1" "AAACAGCCACTTGTTC-1" "AAACAGCCAGAGAGCC-1"


Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from atac_comb_ to ataccomb_”


[1] "2023-11-08 16:07:28 PST"

In [35]:
    #### Built feature matrices
    counts <- FeatureMatrix(
      fragments = frags,
      features = combined.peaks,
      cells = rownames(sobj@meta.data)
    )

Extracting reads overlapping genomic regions



In [13]:
sobj <- li[[1]]
frags <- Fragments(sobj)
frags

[[1]]
A Fragment object for 20000 cells


In [20]:
head(sobj@meta.data)

,BARCODE,orig.ident,nCount_RNA,nFeature_RNA,nCount_ATAC,nFeature_ATAC,fragments_freq_count,FRiP,percent.mt,nCount_SCT,⋯,BEST.POSTERIOR,SNG.POSTERIOR,SNG.BEST.GUESS,SNG.BEST.LLK,SNG.NEXT.GUESS,SNG.NEXT.LLK,SNG.ONLY.POSTERIOR,DBL.BEST.GUESS,DBL.BEST.LLK,DIFF.LLK.SNG.DBL
,<chr>,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,AAACAGCCAAACGGGC-1,QY_2225_2_QY_2224_2,6581,3072,5854,2926,9272,0.6313632,1.1092539,6172,⋯,-2.5e+02,1,UCSDX51_UCSDX51,-243.32,D371_D371,-574.97,0,"D381_D381,UCSDX51_UCSDX51,0.50",-301.66,58.34
2,AAACAGCCAAAGGTAC-1,QY_2225_2_QY_2224_2,6785,2512,2357,1209,3871,0.6088866,3.3898305,6013,⋯,-1.8e+02,1,D265_D265,-173.03,UCSDX3_UCSDX3,-291.65,0,"D265_D265,UCSDX3_UCSDX3,0.50",-182.59,9.56
3,AAACAGCCAAATACCT-1,QY_2225_2_QY_2224_2,773,607,2335,1213,3731,0.6258376,1.1642950,4272,⋯,-7.2e+01,1,UCSDX19_UCSDX19,-68.06,UCSDX51_UCSDX51,-145.11,0,"UCSDX19_UCSDX19,UCSDX31_UCSDX31,0.50",-79.20,11.14
4,AAACAGCCAATTGACT-1,QY_2225_2_QY_2224_2,1709,1121,8280,4112,20682,0.4003481,2.5746050,4407,⋯,-4.0e+02,1,UCSDX42_UCSDX42,-391.82,UCSDX40_UCSDX40,-956.91,0,"UCSDX40_UCSDX40,UCSDX42_UCSDX42,0.50",-486.25,94.43
5,AAACAGCCACATAGCC-1,QY_2225_2_QY_2224_2,5791,2752,84,43,81,1.0370370,0.4144362,5755,⋯,-5.0e+01,1,D339_D339,-45.57,UCSDX2_UCSDX2,-129.87,0,"D339_D339,D371_D371,0.50",-62.54,16.96
6,AAACAGCCACCAGCAT-1,QY_2225_2_QY_2224_2,13831,4553,7542,3804,9703,0.7772854,0.6217916,6228,⋯,5.0e-207,1,UCSDX44_UCSDX44,-625.03,UCSDX19_UCSDX19,-680.52,0,"UCSDX19_UCSDX19,UCSDX44_UCSDX44,0.50",-468.25,-156.78


In [23]:
rownames(sobj@meta.data) <- sobj$BARCODE

In [24]:
head(sobj@meta.data)

,BARCODE,orig.ident,nCount_RNA,nFeature_RNA,nCount_ATAC,nFeature_ATAC,fragments_freq_count,FRiP,percent.mt,nCount_SCT,⋯,BEST.POSTERIOR,SNG.POSTERIOR,SNG.BEST.GUESS,SNG.BEST.LLK,SNG.NEXT.GUESS,SNG.NEXT.LLK,SNG.ONLY.POSTERIOR,DBL.BEST.GUESS,DBL.BEST.LLK,DIFF.LLK.SNG.DBL
,<chr>,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
AAACAGCCAAACGGGC-1,AAACAGCCAAACGGGC-1,QY_2225_2_QY_2224_2,6581,3072,5854,2926,9272,0.6313632,1.1092539,6172,⋯,-2.5e+02,1,UCSDX51_UCSDX51,-243.32,D371_D371,-574.97,0,"D381_D381,UCSDX51_UCSDX51,0.50",-301.66,58.34
AAACAGCCAAAGGTAC-1,AAACAGCCAAAGGTAC-1,QY_2225_2_QY_2224_2,6785,2512,2357,1209,3871,0.6088866,3.3898305,6013,⋯,-1.8e+02,1,D265_D265,-173.03,UCSDX3_UCSDX3,-291.65,0,"D265_D265,UCSDX3_UCSDX3,0.50",-182.59,9.56
AAACAGCCAAATACCT-1,AAACAGCCAAATACCT-1,QY_2225_2_QY_2224_2,773,607,2335,1213,3731,0.6258376,1.1642950,4272,⋯,-7.2e+01,1,UCSDX19_UCSDX19,-68.06,UCSDX51_UCSDX51,-145.11,0,"UCSDX19_UCSDX19,UCSDX31_UCSDX31,0.50",-79.20,11.14
AAACAGCCAATTGACT-1,AAACAGCCAATTGACT-1,QY_2225_2_QY_2224_2,1709,1121,8280,4112,20682,0.4003481,2.5746050,4407,⋯,-4.0e+02,1,UCSDX42_UCSDX42,-391.82,UCSDX40_UCSDX40,-956.91,0,"UCSDX40_UCSDX40,UCSDX42_UCSDX42,0.50",-486.25,94.43
AAACAGCCACATAGCC-1,AAACAGCCACATAGCC-1,QY_2225_2_QY_2224_2,5791,2752,84,43,81,1.0370370,0.4144362,5755,⋯,-5.0e+01,1,D339_D339,-45.57,UCSDX2_UCSDX2,-129.87,0,"D339_D339,D371_D371,0.50",-62.54,16.96
AAACAGCCACCAGCAT-1,AAACAGCCACCAGCAT-1,QY_2225_2_QY_2224_2,13831,4553,7542,3804,9703,0.7772854,0.6217916,6228,⋯,5.0e-207,1,UCSDX44_UCSDX44,-625.03,UCSDX19_UCSDX19,-680.52,0,"UCSDX19_UCSDX19,UCSDX44_UCSDX44,0.50",-468.25,-156.78


In [ ]:
# metadata from rownames removed and turned to numbers
# cell labels should be rownames(sobj@meta.data ) but are now only in BARCODE

In [22]:
head(sobj$BARCODE)

AAACAGCCAAACGGGC-1   AAACAGCCAAAGGTAC-1   AAACAGCCAAATACCT-1 
"AAACAGCCAAACGGGC-1" "AAACAGCCAAAGGTAC-1" "AAACAGCCAAATACCT-1" 
  AAACAGCCAATTGACT-1   AAACAGCCACATAGCC-1   AAACAGCCACCAGCAT-1 
"AAACAGCCAATTGACT-1" "AAACAGCCACATAGCC-1" "AAACAGCCACCAGCAT-1"

In [38]:
counts <- FeatureMatrix(
      fragments = frags,
      features = combined.peaks,
      cells = rownames(sobj@meta.data)
    )

Extracting reads overlapping genomic regions



In [36]:
dim(counts)

NULL

In [37]:
counts

NULL

In [ ]:
assay <- CreateChromatinAssay(counts, fragments = frags)


In [16]:
combined.peaks

GRanges object with 172997 ranges and 0 metadata columns:
             seqnames          ranges strand
                <Rle>       <IRanges>  <Rle>
       [1]       chr1      9767-10709      *
       [2]       chr1   180707-181668      *
       [3]       chr1   183903-184782      *
       [4]       chr1   186352-187360      *
       [5]       chr1   191053-191961      *
       ...        ...             ...    ...
  [172993] KI270728.1 1792043-1792755      *
  [172994] KI270731.1       4419-5408      *
  [172995] KI270734.1   116882-117921      *
  [172996] KI270734.1   121024-121925      *
  [172997] KI270734.1   163727-164637      *
  -------
  seqinfo: 37 sequences from an unspecified genome; no seqlengths

In [15]:
counts

NULL

In [9]:
combined.peaks

GRanges object with 172997 ranges and 0 metadata columns:
             seqnames          ranges strand
                <Rle>       <IRanges>  <Rle>
       [1]       chr1      9767-10709      *
       [2]       chr1   180707-181668      *
       [3]       chr1   183903-184782      *
       [4]       chr1   186352-187360      *
       [5]       chr1   191053-191961      *
       ...        ...             ...    ...
  [172993] KI270728.1 1792043-1792755      *
  [172994] KI270731.1       4419-5408      *
  [172995] KI270734.1   116882-117921      *
  [172996] KI270734.1   121024-121925      *
  [172997] KI270734.1   163727-164637      *
  -------
  seqinfo: 37 sequences from an unspecified genome; no seqlengths

In [73]:
pmo_list

[[1]]
An object of class Seurat 
311642 features across 20000 samples within 4 assays 
Active assay: ATAC (112851 features, 0 variable features)
 3 other assays present: RNA, SCT, ATAC_comb
 2 dimensional reductions calculated: pca, umap

[[2]]
An object of class Seurat 
313329 features across 20000 samples within 4 assays 
Active assay: ATAC (114468 features, 0 variable features)
 3 other assays present: RNA, SCT, ATAC_comb
 2 dimensional reductions calculated: pca, umap


In [75]:
experiments <- extract_samp_nm(pmo_list)

#proj <- "230821_multiome_merge_liver"

print("Removing ATAC assay")
# remove atac assay
for (i in 1:length(pmo_list)) {
    print(i)
    DefaultAssay(pmo_list[[i]]) <- 'ATAC_comb'
    print(pmo_list[[i]])
    pmo_list[[i]][['ATAC']]<- NULL
}



[1] "Removing ATAC assay"
[1] 1
An object of class Seurat 
311642 features across 20000 samples within 4 assays 
Active assay: ATAC_comb (131874 features, 0 variable features)
 3 other assays present: RNA, ATAC, SCT
 2 dimensional reductions calculated: pca, umap
[1] 2
An object of class Seurat 
313329 features across 20000 samples within 4 assays 
Active assay: ATAC_comb (131874 features, 0 variable features)
 3 other assays present: RNA, ATAC, SCT
 2 dimensional reductions calculated: pca, umap


In [76]:
for (i in 1:length(pmo_list)) {
    print(i)
    print(pmo_list[[i]])
}

[1] 1
An object of class Seurat 
198791 features across 20000 samples within 3 assays 
Active assay: ATAC_comb (131874 features, 0 variable features)
 2 other assays present: RNA, SCT
 2 dimensional reductions calculated: pca, umap
[1] 2
An object of class Seurat 
198861 features across 20000 samples within 3 assays 
Active assay: ATAC_comb (131874 features, 0 variable features)
 2 other assays present: RNA, SCT
 2 dimensional reductions calculated: pca, umap


In [77]:
experiments

[1] "QY_2225_2_QY_2224_2" "QY_2227_2_QY_2226_2"

In [78]:
print("Merging experiments")
# adding experiments (library_id) to keep cell names unique 
Sys.time()
all.objs <- merge(
  x = pmo_list[[1]],
  y = pmo_list[2:length(pmo_list)],
       add.cell.ids = experiments,
                project = proj
)

[1] "Merging experiments"


[1] "2023-11-08 16:09:59 PST"

In [79]:
all.objs

An object of class Seurat 
199502 features across 40000 samples within 3 assays 
Active assay: ATAC_comb (131874 features, 0 variable features)
 2 other assays present: RNA, SCT

In [ ]:
print(pmo_list) # now they all have the same number of features, using the same combined.peaks


experiments <- extract_samp_nm(pmo_list)